In [1]:
import numpy as np, nnfs, matplotlib.pyplot as plt
from nnfs.datasets import spiral_data

nnfs.init()

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weight_regulariser_l1=0, weight_regulariser_l2=0, bias_regulariser_l1=0, bias_regulariser_l2=0):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        self.weight_regulariser_l1 = weight_regulariser_l1
        self.weight_regulariser_l2 = weight_regulariser_l2
        self.bias_regulariser_l1 = bias_regulariser_l1
        self.bias_regulariser_l2 = bias_regulariser_l2

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, 0, keepdims=True)
        if self.weight_regulariser_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regulariser_l1 * dL1
        if self.weight_regulariser_l2 > 0: self.dweights += 2 * self.weight_regulariser_l2 * self.weights
        if self.bias_regulariser_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regulariser_l1 * dL1
        if self.bias_regulariser_l2 > 0: self.dbiases += 2 * self.bias_regulariser_l2 * self.biases
        self.dinputs = np.dot(dvalues, self.weights.T)

class Layer_Dropout:
    def __init__(self, rate):
        self.rate = 1 - rate

    def forward(self, inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1, self.rate, inputs.shape) / self.rate
        self.output = inputs * self.binary_mask
    
    def backward(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

In [3]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, 1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, 1, keepdims=True)

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for i, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            self.dinputs[i] = np.dot(np.diagflat(single_output) - np.dot(single_output, single_output.T), single_dvalues)

class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output

In [4]:
class Optimiser_SGD:
    def __init__(self, lr=1., decay=0., momentum=0.):
        self.lr = lr
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay: self.lr /= 1 + self.decay * self.iterations
    
    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            weight_updates = self.momentum * layer.weight_momentums - self.lr * layer.dweights
            bias_updates = self.momentum * layer.bias_momentums - self.lr * layer.dbiases
        else:
            weight_updates = -self.lr * layer.dweights
            bias_updates = -self.lr * layer.dbiases
        layer.weights += weight_updates
        layer.biases += bias_updates
    
    def post_update_params(self):
        self.iterations += 1

class Optimiser_Adagrad:
    def __init__(self, lr=1., decay=0., epsilon=1e-7):
        self.lr = lr
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def pre_update_params(self):
        if self.decay: self.lr /= 1 + self.decay * self.iterations
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        layer.weights -= self.lr * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases -= self.lr * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
    
    def post_update_params(self):
        self.iterations += 1

class Optimiser_RMSprop:
    def __init__(self, lr=0.001, decay=0., epsilon=1e-7, rho=0.9):
        self.lr = lr
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho

    def pre_update_params(self):
        if self.decay: self.lr /= 1 + self.decay * self.iterations
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases**2
        layer.weights -= self.lr * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases -= self.lr * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
    
    def post_update_params(self):
        self.iterations += 1

class Optimiser_Adam:
    def __init__(self, lr=0.001, decay=0., epsilon=1e-7, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta1 = beta1
        self.beta2 = beta2

    def pre_update_params(self):
        if self.decay: self.lr /= 1 + self.decay * self.iterations
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = layer.bias_cache = np.zeros_like(layer.biases)
        layer.weight_momentums = self.beta1 * layer.weight_momentums + (1 - self.beta1) * layer.dweights
        layer.bias_momentums = self.beta1 * layer.bias_momentums + (1 - self.beta1) * layer.dbiases
        layer.weight_cache = self.beta2 * layer.weight_cache + (1 - self.beta2) * layer.dweights**2
        layer.bias_cache = self.beta2 * layer.bias_cache + (1 - self.beta2) * layer.dbiases**2
        layer.weights -= self.lr * layer.weight_momentums / ((1 - self.beta1**(self.iterations + 1)) * (np.sqrt(layer.weight_cache / (1 - self.beta2**(self.iterations + 1))) + self.epsilon))
        layer.biases -= self.lr * layer.bias_momentums / ((1 - self.beta1**(self.iterations + 1)) * (np.sqrt(layer.bias_cache / (1 - self.beta2**(self.iterations + 1))) + self.epsilon))
    
    def post_update_params(self):
        self.iterations += 1

In [5]:
class Loss:
    def regularisation_loss(self, layer):
        regularisation_loss = 0
        if layer.weight_regulariser_l1 > 0: regularisation_loss += layer.weight_regulariser_l1 * np.sum(np.abs(layer.weights))
        if layer.weight_regulariser_l2 > 0: regularisation_loss += layer.weight_regulariser_l2 * np.sum(layer.weights**2)
        if layer.bias_regulariser_l1 > 0: regularisation_loss += layer.bias_regulariser_l1 * np.sum(np.abs(layer.biases))
        if layer.bias_regulariser_l2 > 0: regularisation_loss += layer.bias_regulariser_l2 * np.sum(layer.biases**2)
        return regularisation_loss

    def calculate(self, output, y):
        return np.mean(self.forward(output, y))

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1: correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]
        elif len(y_true.shape) == 2: correct_confidences = np.sum(y_pred_clipped * y_true, 1)
        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        if len(y_true.shape) == 1: y_true = np.eye(len(dvalues[0]))[y_true]
        self.dinputs = -y_true / (dvalues * len(dvalues))

class Activation_Softmax_Loss_CategoricalCrossentropy:
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2: y_true = np.argmax(y_true, 1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs /= samples

class Loss_BinaryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        return np.mean(-np.log(y_pred_clipped**y_true * (1 - y_pred_clipped)**(1 - y_true)), -1)
    
    def backward(self, dvalues, y_true):
        clipped_values = np.clip(dvalues, 1e-7, 1 - 1e-7)
        self.dinputs = ((y_true - 1) / (clipped_values - 1) - y_true / clipped_values) / (len(dvalues[0]) * len(dvalues))

In [ ]:
X, y = spiral_data(100, 3)
y = y.reshape(-1, 1)
dense1 = Layer_Dense(2, 64, weight_regulariser_l2=5e-4, bias_regulariser_l2=5e-4)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 1)
activation2 = Activation_Sigmoid()
loss_function = Loss_BinaryCrossentropy()
optimiser = Optimiser_Adam(decay=5e-7)
for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    data_loss = loss_function.calculate(activation2.output, y)
    regularisation_loss = loss_function.regularisation_loss(dense1) + loss_function.regularisation_loss(dense2)
    loss = data_loss + regularisation_loss
    preds = activation2 > 0.5
    acc = np.mean(preds == y)
    if not epoch % 100: print(f'epoch: {epoch}, acc: {acc:.3f}, loss: {(data_loss + regularisation_loss):.3f} ')